In [ ]:
import pandas as pd
import dns.resolver

def verificar_registros_mx(dominio):
    try:
        # Consultar registros MX del dominio
        records = dns.resolver.resolve(dominio, 'MX')
        mx_records = [record.exchange.to_text() for record in records]
        
        if mx_records:
            return "valido"
        else:
            return "invalido"
    
    except dns.resolver.NoAnswer:
        return "invalido"
    except dns.resolver.NXDOMAIN:
        return "invalido"
    except Exception as e:
        return f"error: {str(e)}"

# Leer el archivo CSV
archivo_csv = 'Emails.csv'  # Reemplaza con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)

# Verificar cada dominio y actualizar el DataFrame
df['Domain Status'] = df['Dominio'].apply(verificar_registros_mx)

# Guardar el DataFrame actualizado en el mismo archivo CSV
df.to_csv(archivo_csv, index=False)

print("Verificación de dominios completada y archivo CSV actualizado.")


In [ ]:
import pandas as pd

def generar_posibles_correos(nombre, apellido, dominio):
    posibles_correos = []
    
    # Generar combinaciones básicas
    posibles_correos.append(f"{nombre}@{dominio}")
    posibles_correos.append(f"{apellido}@{dominio}")
    posibles_correos.append(f"{nombre}.{apellido}@{dominio}")
    posibles_correos.append(f"{nombre}{apellido}@{dominio}")
    posibles_correos.append(f"{nombre[0]}{apellido}@{dominio}")
    posibles_correos.append(f"{nombre}{apellido[0]}@{dominio}")
    posibles_correos.append(f"{nombre}_{apellido}@{dominio}")
    posibles_correos.append(f"{apellido}_{nombre}@{dominio}")
    posibles_correos.append(f"{nombre}-{apellido}@{dominio}")
    posibles_correos.append(f"{apellido}-{nombre}@{dominio}")
    
    # Otras combinaciones posibles
    if len(nombre) >= 2:
        posibles_correos.append(f"{nombre[0]}.{apellido}@{dominio}")
        posibles_correos.append(f"{nombre}.{apellido[0]}@{dominio}")
    if len(apellido) >= 2:
        posibles_correos.append(f"{nombre[0]}_{apellido}@{dominio}")
        posibles_correos.append(f"{nombre}_{apellido[0]}@{dominio}")
    
    return posibles_correos[:10]  # Limitar a los primeros 10 posibles correos

# Leer el archivo CSV
archivo_csv = 'Emails.csv'  # Reemplaza con el nombre de tu archivo CSV
df = pd.read_csv(archivo_csv)

# Generar posibles correos y actualizar el DataFrame
df['Posibles Emails'] = df.apply(lambda row: generar_posibles_correos(row['Nombre'], row['Apellido'], row['Dominio']), axis=1)

# Convertir la lista de posibles correos en una cadena para cada fila
df['Posibles Emails'] = df['Posibles Emails'].apply(lambda lista: ', '.join(lista))

# Guardar el DataFrame actualizado en el mismo archivo CSV
df.to_csv(archivo_csv, index=False)

print("Generación de posibles correos y archivo CSV actualizado.")


In [ ]:
import pandas as pd
import dns.resolver
import smtplib

def verificar_correo_puede_recibir(email):
    dominio = email.split('@')[1]

    # Verificar si el dominio tiene registros MX
    try:
        registros_mx = dns.resolver.resolve(dominio, 'MX')
    except dns.resolver.NoAnswer:
        print(f"No se encontraron registros MX para {dominio}")
        return False
    except dns.resolver.NXDOMAIN:
        print(f"Dominio {dominio} no encontrado")
        return False
    except Exception as e:
        print(f"Error al verificar el dominio {dominio}: {str(e)}")
        return False
    
    # Intentar conectarse al servidor SMTP del dominio
    for mx in registros_mx:
        mx_servidor = str(mx.exchange)
        try:
            with smtplib.SMTP(mx_servidor) as server:
                server.set_debuglevel(0)  # Establecer a 1 para ver más detalles de la conexión

                # Comando HELO
                server.helo("example.com")

                # Simulación de envío de correo
                server.mail("Gabrielbg21@hotmail.com")
                response_code, response_message = server.rcpt(email)
                if response_code == 250 or response_code == 251:
                    server.quit()
                    return True
                else:
                    print(f"Servidor respondió con {response_code} {response_message}")

                # Comando QUIT
                server.quit()

        except smtplib.SMTPConnectError as e:
            print(f"No se pudo conectar al servidor SMTP de {mx_servidor}: {str(e)}")
        except Exception as e:
            print(f"Error inesperado al verificar {email} con {mx_servidor}: {str(e)}")
    
    return False

# Leer el CSV
df = pd.read_csv('Emails.csv')

# Crear una lista para almacenar los correos válidos
correos_validos = []

# Iterar sobre cada fila del DataFrame
for index, row in df.iterrows():
    posibles_emails = row['Posibles Emails'].split(', ')
    correo_valido = None
    for email in posibles_emails:
        if verificar_correo_puede_recibir(email):
            correo_valido = email
            break
    correos_validos.append(correo_valido)

# Añadir la nueva columna al DataFrame
df['Email Valid'] = correos_validos

# Sobrescribir el CSV con la nueva columna
df.to_csv('Emails.csv', index=False)

print("Archivo CSV actualizado con los correos válidos.")


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googlesearch import search

def obtener_telefono_desde_pagina(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Intenta buscar un patrón común de número de teléfono
        telefono_tag = soup.find_all(string=lambda text: any(keyword in text for keyword in ["Tel", "tel", "Phone", "phone"]))
        for tag in telefono_tag:
            parent = tag.parent
            if parent:
                phone = parent.get_text(strip=True)
                if phone:
                    return phone
        return "No encontrado"
    except Exception as e:
        return f"Error: {str(e)}"

def buscar_telefono_en_google(dominio):
    query = f'site:{dominio} contacto OR contact OR about OR nosotros OR support OR ayuda'
    urls = list(search(query, num_results=5))  # Limitar a los primeros 5 resultados
    for url in urls:
        telefono = obtener_telefono_desde_pagina(url)
        if telefono != "No encontrado":
            return telefono
    return "No encontrado"

# Leer el archivo CSV
archivo_csv = 'Emails.csv'
df = pd.read_csv(archivo_csv)

# Verificar cada dominio y actualizar el DataFrame
df['Telefono'] = df['Dominio'].apply(buscar_telefono_en_google)

# Guardar el DataFrame actualizado en el mismo archivo CSV
df.to_csv(archivo_csv, index=False)

print("Extracción de números de teléfono completada y archivo CSV actualizado.")


Extracción de números de teléfono completada y archivo CSV actualizado.
